# Life Cycle Impact Assessment (LCIA)

This notebook performs the impact assessment calculations and export the results

In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import pickle
import datetime

from utils import(
    init_simple_lca,
    mlca,
    recursive_calculation_cumulative_flows,
)

SCENARIO_DATA_PATH = Path("../scenario_data")
RESULTS_PATH = Path("../results")

## 1. Basic set-up

In [2]:
# Open BW project
BW_PROJECT = 'lithium_brine'
bw.projects.set_current(BW_PROJECT)

# Databases
BIO_DBs = ["ecoinvent-3.10-biosphere", "biosphere water regionalized"]
LI_PROJECTS_DB = "lithium_brine_projects"
dbs = [db for db in bw.databases if "lithium_S&P" in db] # List of prospective databases

# List of lithium carbonate market activities
markets_inv = {
    ds.key: (ds["name"], ds["reference product"], ds["location"])
    for db in dbs for ds in bw.Database(db) if ds["name"] == 'market for lithium carbonate, from brine, S&P forecasts'
}

In [3]:
# Import lithium scenario results
lithium_scenario_data = pd.read_csv(SCENARIO_DATA_PATH / "external" / "lithium_scenario_data_22-01-2025.csv", index_col=0)
lithium_production_by_country = pd.read_csv(RESULTS_PATH / "lithium_production_by_country_22-01-2025.csv", index_col=0)
lithium_production_total = pd.read_csv(RESULTS_PATH / "lithium_production_total_22-01-2025.csv", index_col=0)

# Import map of lithium project names to LCI datasets
map_lithium_projects_to_lci = pd.read_excel(SCENARIO_DATA_PATH / "map_litihum_projects_to_lci.xlsx")

# Import annonymized IDs for projects
with open(SCENARIO_DATA_PATH / "external" / "anonymized_projects_id.yaml", "r") as yaml_file:
    anonymized_projects_id = yaml.safe_load(yaml_file)

# List of products for breakdown analysis
with open("breakdown_lists.yaml", "r") as yaml_file:
    breakdown_lists = yaml.safe_load(yaml_file)

# List of countries, scenarios and years
COUNTRIES = list(set([entry[2] for entry in markets_inv.values()]))
LI_SCENARIOS = ["S&P-Baseline", "S&P-Ambitious", "S&P-Very Ambitious"]

# Impact assessment methods
LCIA_METHODS = {
        'Climate change': ('IPCC 2021', "climate change", 'GWP 100a, incl. H and bio CO2'),
        'Water scarcity': ('AWARE regionalized', 'Annual', 'All')
}

In [4]:
scenario_db_mapping = {
    "S&P-Baseline": {
        "SSP2-NDC": {
            2020: 'lithium_S&P-Baseline_remind SSP2-NDC 2020',
            2025: 'lithium_S&P-Baseline_remind SSP2-NDC 2025',
            2030: 'lithium_S&P-Baseline_remind SSP2-NDC 2030',
            2035: 'lithium_S&P-Baseline_remind SSP2-NDC 2035'
        },
        "SSP2-2°C": {
            2025: 'lithium_S&P-Baseline_remind SSP2-PkBudg1150 2025',
            2030: 'lithium_S&P-Baseline_remind SSP2-PkBudg1150 2030',
            2035: 'lithium_S&P-Baseline_remind SSP2-PkBudg1150 2035'
        }
    },

    "S&P-Ambitious": {
        "SSP2-NDC": {
            2025: 'lithium_S&P-Ambitious_remind SSP2-NDC 2025',
            2030: 'lithium_S&P-Ambitious_remind SSP2-NDC 2030',
            2035: 'lithium_S&P-Ambitious_remind SSP2-NDC 2035'
        },
        "SSP2-2°C": {
            2025: 'lithium_S&P-Ambitious_remind SSP2-PkBudg1150 2025',
            2030: 'lithium_S&P-Ambitious_remind SSP2-PkBudg1150 2030',
            2035: 'lithium_S&P-Ambitious_remind SSP2-PkBudg1150 2035'
        }
    },

    "S&P-Very Ambitious": {
        "SSP2-NDC": {
            2025: 'lithium_S&P-Very Ambitious_remind SSP2-NDC 2025',
            2030: 'lithium_S&P-Very Ambitious_remind SSP2-NDC 2030',
            2035: 'lithium_S&P-Very Ambitious_remind SSP2-NDC 2035'
        },
        "SSP2-2°C": {
            2025: 'lithium_S&P-Very Ambitious_remind SSP2-PkBudg1150 2025',
            2030: 'lithium_S&P-Very Ambitious_remind SSP2-PkBudg1150 2030',
            2035: 'lithium_S&P-Very Ambitious_remind SSP2-PkBudg1150 2035'
        }          
    }
}

## 1. LCI indicators (Li concentration, electricity/heat/reagents/freshwater intensity)

In [5]:
# Calculate cumulative LCI flows for each project
projects_lci_indicators = []

for index, proj in map_lithium_projects_to_lci.iterrows():
    # Skip projects without site-specifc LCIs
    if proj["LCI dataset"] == "Other projects":
        continue
    
    # Identify the activity and project name based on the LCI dataset format
    if "df_rotary_dryer_" in proj["LCI dataset"]:
        activity = [a for a in bw.Database(LI_PROJECTS_DB) if a["name"]==proj["LCI dataset"]][0]
        project_lci_name = proj["LCI dataset"].split("df_rotary_dryer_")[1]
    else:
        raise ValueError(f"Unexpected LCI dataset format: {proj['LCI dataset']}")

    # Get foreground processes for the project
    foreground_processes = [a for a in bw.Database(LI_PROJECTS_DB) if project_lci_name in a["name"]]
    if not foreground_processes:
        raise KeyError(f"Project-related datasets not found for project: {project_lci_name}")

    # Calculate project cumulative LCI flows
    project_cumulative_flows = recursive_calculation_cumulative_flows(
        activity, foreground_processes, amount=1)
    
    # Group LCI flows into the defined LCI indicators:
    project_indicators = {indicator: 0 for indicator in list(breakdown_lists.keys())}
    for exc, flow_amount in project_cumulative_flows.items():
        for indicator, relevant_items in breakdown_lists.items():
            if (
                (exc["type"] == "process" and exc.get("reference product") in relevant_items) or 
                (exc["type"] != "process" and exc["name"] in relevant_items)
            ):
                project_indicators[indicator] += flow_amount
    
    # Append LCI indicators together with project name, LCI dataset, and Li concentration
    projects_lci_indicators.append((proj["Project name"], proj["LCI dataset"], proj["Li concentration"]) + tuple(project_indicators.values()))

# Convert to dataframe
projects_lci_indicators = pd.DataFrame(projects_lci_indicators, 
                                  columns=["Project name", "LCI dataset", "Li concentration", "Electricity intensity",
                                            "Heat intensity", "Reagent intensity", "Freshwater intensity"])
# Covert m3 to kg
projects_lci_indicators["Freshwater intensity"] *= 1000

# Export project-specific LCI indicators results
projects_lci_indicators.to_csv(RESULTS_PATH / f"SI_projects_LCI_indicators_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

In [6]:
# Calculate global average indicators based on project-specific performance and market shares
performance_indicators = []

for li_sc in scenario_db_mapping:
    for year in [2020, 2025, 2030, 2035]:
        li_concentration = 0
        electricity_intensity = 0
        heat_intensity = 0
        reagent_intensity = 0
        freshwater_intensity = 0
       
        # Get global market for lithium production
        if year == 2020:
            db = scenario_db_mapping['S&P-Baseline']['SSP2-NDC'][2020]
        else:
            db = scenario_db_mapping[li_sc]["SSP2-NDC"][year]
        glo_market = [a for a in bw.Database(db) if a["name"]=="market for lithium carbonate, from brine, S&P forecasts" 
                                           and a["location"]=="World"][0]
        
        # Iterate over country markets
        for c in glo_market.technosphere():
            country_amount = c["amount"]
            c_ds = [a for a in bw.Database(db) if a.key==c["input"]][0]
            
            # Iterate over projects within each ountry
            for proj in c_ds.technosphere():
                project_market_share = country_amount * proj["amount"]

                # If dataset is a specific project
                if proj["name"] in list(projects_lci_indicators["LCI dataset"]):
                    li_concentration += project_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj["name"]]["Li concentration"].values[0] * 100
                    electricity_intensity += project_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj["name"]]["Electricity intensity"].values[0]
                    heat_intensity += project_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj["name"]]["Heat intensity"].values[0]
                    reagent_intensity += project_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj["name"]]["Reagent intensity"].values[0]
                    freshwater_intensity += project_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj["name"]]["Freshwater intensity"].values[0]

                # If dataset is the country-average market:
                else:
                    # Get country average dataset
                    avg_ds = [a for a in bw.Database(db) if a.key==proj["input"]][0]
                    # Iterate over projects within average dataset:
                    for proj_avg in avg_ds.technosphere():
                        project_avg_market_share = project_market_share * proj_avg["amount"]

                        li_concentration += project_avg_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj_avg["name"]]["Li concentration"].values[0] * 100
                        electricity_intensity += project_avg_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj_avg["name"]]["Electricity intensity"].values[0]
                        heat_intensity += project_avg_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj_avg["name"]]["Heat intensity"].values[0]
                        reagent_intensity += project_avg_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj_avg["name"]]["Reagent intensity"].values[0]
                        freshwater_intensity += project_avg_market_share * projects_lci_indicators[projects_lci_indicators["LCI dataset"]==proj_avg["name"]]["Freshwater intensity"].values[0]

        performance_indicators.append((li_sc, year, li_concentration, electricity_intensity, heat_intensity, reagent_intensity, freshwater_intensity))

performance_indicators = pd.DataFrame(performance_indicators, columns=["Lithium scenario", "Year", 
                                    "Li concentration", "Electricity intensity", "Heat intensity", "Reagent intensity", "Freshwater intensity"])

In [7]:
# Export contribution analysis
performance_indicators.to_csv(RESULTS_PATH / f"performance_indicators_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

## 2 Calculate impacts per project for sanity-check

In [8]:
projects_inv = [ds for ds in bw.Database("lithium_brine_projects") if "df_rotary_dryer" in ds["name"]]

calculation_setup = {}
calculation_setup["inventories"] = [{(ds["database"], ds["code"]): 1} for ds in projects_inv]
calculation_setup["methods"] = list(LCIA_METHODS.values())

lca = bw.LCA(demand=calculation_setup["inventories"][0], method=calculation_setup["methods"][0])
lca = init_simple_lca(calculation_setup)
lca_results_projects = mlca(lca, calculation_setup)

In [14]:
lca_results_projects_df = {}
for ds in lca_results_projects:
    activity = bw.Database("lithium_brine_projects").get(ds[0][1])
    project_name = activity["name"].split("df_rotary_dryer_")[1]
    lca_results_projects_df.update({project_name: lca_results_projects[ds]})

lca_results_projects_df = pd.DataFrame(lca_results_projects_df).T

In [15]:
lca_results_projects_df

,IPCC 2021,AWARE regionalized
,climate change,Annual
,"GWP 100a, incl. H and bio CO2",All
Salar de Antofalla,35.525402,10.439603
Salar de Tolillar,36.093296,12.958634
Salar de Centenario,34.211247,13.262947
Maricunga,7.975822,17.759883
Pozuelos,52.913209,43.377774
Fenix,30.517773,10.779307
Chaerhan,40.082144,11.309119
Salar de Atacama,3.976847,6.080192


## 2. Impacts by country (intensity + total)

In [9]:
# Calculate impacts for all lithium market datasets from each database
lca_results_country = {}

for db in dbs:
    print("Calculating impacts for scenario/database:", db)

    # Get lithium market datasets
    markets = [key for key, value in markets_inv.items() if key[0] == db]
    activities = [a for a in bw.Database(db) if a.key in markets]

    # Define calculation setup
    calculation_setup = {}
    calculation_setup["inventories"] = [{ds.key: 1} for ds in activities]
    calculation_setup["methods"] = list(LCIA_METHODS.values())

    # Initialize LCA object and calculate impacts
    lca = bw.LCA(demand=calculation_setup["inventories"][0], method=calculation_setup["methods"][0])
    lca = init_simple_lca(calculation_setup)
    lca_results_country[db] = mlca(lca, calculation_setup)

    print("Done!")

Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-NDC 2020
Done!
Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-PkBudg1150 2025
Done!
Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-NDC 2025
Done!
Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-PkBudg1150 2030
Done!
Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-NDC 2030
Done!
Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-PkBudg1150 2035
Done!
Calculating impacts for scenario/database: lithium_S&P-Baseline_remind SSP2-NDC 2035
Done!
Calculating impacts for scenario/database: lithium_S&P-Ambitious_remind SSP2-PkBudg1150 2025
Done!
Calculating impacts for scenario/database: lithium_S&P-Ambitious_remind SSP2-NDC 2025
Done!
Calculating impacts for scenario/database: lithium_S&P-Ambitious_remind SSP2-PkBudg1150 2030
Done!
Calculating impacts for scenario/database: lithium_S

In [16]:
# Export raw LCA results by country to a pickle file
with open(RESULTS_PATH / "raw" / f"lca_results_country_raw_{datetime.datetime.today().strftime('%d-%m-%Y')}.pkl", 'wb') as f:
    pickle.dump(lca_results_country, f)

# Load LCA results by country
#with open(RESULTS_PATH / f"raw/lca_results_country_raw_25-02-2025.pkl", 'rb') as f:
#    lca_results_country = pickle.load(f)

In [17]:
# Calculate impacts intensity and total by country and global (kg CO2-eq/kg & Mt CO2-eq/year)
impacts_by_country = []

for impact in LCIA_METHODS:
    for li_sc in scenario_db_mapping:
        for iam_sc in scenario_db_mapping[li_sc]:
            for year in [2020, 2025, 2030, 2035]:
                
                # Get global production for scenario and year (in kg/year)
                glo_production = lithium_production_total.loc[li_sc][str(year)] * 1e6

                # Get global market for lithium production
                if year == 2020:
                    db = scenario_db_mapping['S&P-Baseline']['SSP2-NDC'][2020]
                else:
                    db = scenario_db_mapping[li_sc][iam_sc][year]
                glo_market = [a for a in bw.Database(db) if a["name"]=="market for lithium carbonate, from brine, S&P forecasts" 
                                                            and a["location"]=="World"][0]
                
                # Calculate impact intensity and total for the world
                impact_intensity_glo = lca_results_country[db][(glo_market.key, None)][LCIA_METHODS[impact]]
                impact_total_glo = glo_production * impact_intensity_glo
                impacts_by_country.append((impact, li_sc, iam_sc, year, "World", impact_intensity_glo, impact_total_glo))

                # Calculate impact intensity and total by country
                for country in glo_market.technosphere():
                    country_production = glo_production * country["amount"]
                    country_ds = [a for a in bw.Database(db) if a.key==country["input"]][0]

                    impact_intensity_country = lca_results_country[db][(country_ds.key, None)][LCIA_METHODS[impact]]
                    impact_total_country = country_production * impact_intensity_country
                    impacts_by_country.append((impact, li_sc, iam_sc, year, country_ds["location"], impact_intensity_country, impact_total_country))

impacts_by_country_df = pd.DataFrame(impacts_by_country, columns=['Impact', 'Lithium scenario', "IAM scenario", 'Year', "Country", 'Intensity', "Total"])
# Convert total impact to Mt CO2eq or billion m3
impacts_by_country_df["Total"] /= 1e9

In [18]:
# Export LCA results by country
impacts_by_country_df.to_csv(RESULTS_PATH / f"annual_impacts_by_country_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

# Import results for further analysis
#impacts_by_country_df = pd.read_csv(RESULTS_PATH / "annual_impacts_by_country_25-02-2025.csv")

## 3. Impact by project and contribution analysis

Assessment performed only for the "SSP2-NDC" pathway.

To reduce computational time, the following strategy applies:

1. Cumulative LCI exchanges (technosphere and biosphere) for each project is calculated.
2. A list of all required background (technosphere) datasets is obtained.
3. Calculate impacts per unit of each background dataset.
4. Combine cumulative LCI exchanges with impacts per unit.

In [10]:
# Use the "Very Ambitious" scenario, which contains all the projects
# Note that the impacts of projects do not change across supply scenarios
# Only their market shares change
process_contribution_dbs = {
    2020: scenario_db_mapping['S&P-Baseline']['SSP2-NDC'][2020],
    **scenario_db_mapping['S&P-Very Ambitious']['SSP2-NDC']
}

In [11]:
 # 1. Get cumulative list of exchanges for each project in each database
project_cumulative_flows = {}

for year in process_contribution_dbs:
    db = process_contribution_dbs[year]
    project_cumulative_flows[db] = {}

    for index, proj in map_lithium_projects_to_lci.iterrows():
        # Skip projects without site-specifc LCIs
        if proj["LCI dataset"] == "Other projects":
            continue    

        # Identify the project name and the activity for the correspoding database
        if "df_rotary_dryer_" in proj["LCI dataset"]:
            activity = [a for a in bw.Database(db) if a["name"]==proj["LCI dataset"]][0]
            project_lci_name = proj["LCI dataset"].split("df_rotary_dryer_")[1]
        else:
            raise ValueError(f"Unexpected LCI dataset format: {proj['LCI dataset']}")

        # Get foreground processes for the project
        foreground_processes = [a for a in bw.Database(db) if project_lci_name in a["name"]]
        if not foreground_processes:
            raise KeyError(f"Project-related datasets not found for project: {project_lci_name}")
                
        # Calculate project cumulative LCI flows
        project_cumulative_flows[db][project_lci_name] = recursive_calculation_cumulative_flows(activity, foreground_processes, amount=1)

In [46]:
# 2. Perform LCIA for background datasets

# Create a dictionary of unique background datasets
unique_background_ds = {}
for db, proj in project_cumulative_flows.items():
    unique_background_ds[db] = set()
    for p in proj:
        for f in proj[p]:
            if f["database"] not in BIO_DBs:
                unique_background_ds[db].add(f)

# Perform LCIA of background datasets
lcia_background_datasets = {}
for db in unique_background_ds:
    list_datasets = list(unique_background_ds[db])
    print("Database:", db)
    print("Performing LCIA of background datasets:", len(list_datasets), "datasets")

    # Define calculation setup
    calculation_setup = {}
    calculation_setup["inventories"] = [{ds.key: 1} for ds in list_datasets]
    calculation_setup["methods"] = list(LCIA_METHODS.values())

    # Initialize LCA object and calculate impacts
    lca_background = bw.LCA(demand=calculation_setup["inventories"][0], method=calculation_setup["methods"][0])
    lca_background = init_simple_lca(calculation_setup)
    lca_background_results = mlca(lca_background, calculation_setup)

    lcia_background_datasets[db] = lca_background_results

Database: lithium_S&P-Baseline_remind SSP2-NDC 2020
Performing LCIA of background datasets: 25 datasets
Database: lithium_S&P-Very Ambitious_remind SSP2-NDC 2025
Performing LCIA of background datasets: 25 datasets
Database: lithium_S&P-Very Ambitious_remind SSP2-NDC 2030
Performing LCIA of background datasets: 25 datasets
Database: lithium_S&P-Very Ambitious_remind SSP2-NDC 2035
Performing LCIA of background datasets: 25 datasets


In [ ]:
# Export impacts of background datasets to a pickle file
#with open(RESULTS_PATH / "raw" / f"lca_background_datasets_{datetime.datetime.today().strftime('%d-%m-%Y')}.pkl", 'wb') as f:
#    pickle.dump(lcia_background_datasets, f)

In [12]:
# Load LCA results of background datasets
with open(RESULTS_PATH / f"raw/lca_background_datasets_25-02-2025.pkl", 'rb') as f:
    lcia_background_datasets = pickle.load(f)

In [13]:
# 4. Calculate impact contribution for each project by combining cumulative LCI flows
# with impacts per unit for background datasets and characterization factors for elementary flows
projects_process_contribution = []

for db in project_cumulative_flows:
    for word in db.split():
        if word.isdigit() and len(word) == 4:
            year = int(word)

    for proj in project_cumulative_flows[db]:
        # Group impacts into the defined categories
        # and add contribution from biosphere flows
        for impact_name, impact_method in LCIA_METHODS.items():
            process_contribution = {category: 0 for category in list(breakdown_lists.keys()) + ["Other"]}

            # Group technosphere impacts
            for exc, result in lcia_background_datasets[db].items():
                exc_ds = bw.get_activity(exc[0]) # Get the exchange activity
                try:
                    # Calculate impact of cumulative technosphere exchange
                    exc_impact = project_cumulative_flows[db][proj][exc_ds] * result[impact_method]
                    
                    contribution_added = False
                    for category, category_items in breakdown_lists.items():
                        if exc_ds["reference product"] in category_items:
                            process_contribution[category] += exc_impact
                            contribution_added = True
                    if not contribution_added:
                        process_contribution["Other"] += exc_impact
                except KeyError:
                    pass

            # Add biosphere impacts:
            # Get method's characterization factors
            method_CFs = {ef[0]: ef[1] for ef in bw.Method(impact_method).load()}
            for bio_exc in project_cumulative_flows[db][proj]:
                if bio_exc.key in method_CFs:
                    process_contribution["Foreground"] += project_cumulative_flows[db][proj][bio_exc] * method_CFs[bio_exc.key]

            projects_process_contribution.append((proj, "df_rotary_dryer_"+proj, impact_name, year) + tuple(process_contribution.values()))

projects_process_contribution_df = pd.DataFrame(projects_process_contribution,
                                                columns=["Project name", "LCI dataset", "Impact", "Year"] + [category for category in list(breakdown_lists.keys())] + ["Other"])

# Add total impact intensity for each project
projects_process_contribution_df["Total"] = projects_process_contribution_df[["Electricity", "Heat", "Reagents", "Foreground", "Other"]].sum(axis=1)

In [19]:
# Export project-specific process contribution
projects_process_contribution_df.to_csv(RESULTS_PATH / f"SI_projects_impacts_breakdown_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

# Import results for further analysis
#projects_process_contribution_df = pd.read_csv(RESULTS_PATH / "SI_projects_impacts_breakdown_25-02-2025.csv")

**Impact contribution by technology and impact source**

In [20]:
# Calculate global impacts contribution by technology and impact source
# based on project-specific data and market shares
technology_contribution = {}
process_contribution = []

for li_sc in scenario_db_mapping:
    for year in [2020, 2025, 2030, 2035]:
        # Get global production for scenario and year (in kg/year)
        glo_production = lithium_production_total.loc[li_sc][str(year)] * 1e6

        # Get global market for lithium production
        if year == 2020:
            db = scenario_db_mapping['S&P-Baseline']['SSP2-NDC'][2020]
        else:
            db = scenario_db_mapping[li_sc]["SSP2-NDC"][year]
        glo_market = [a for a in bw.Database(db) if a["name"]=="market for lithium carbonate, from brine, S&P forecasts" 
                                                    and a["location"]=="World"][0]

        for impact in LCIA_METHODS:
            technology_impact_contribution = {}
            
            electricity_contribution = 0
            heat_contribution = 0
            reagent_contribution = 0
            foreground_contribution = 0
            other_contribution = 0
            
            # Iterate over country markets
            for country in glo_market.technosphere():
                country_production = glo_production * country["amount"]
                country_ds = [a for a in bw.Database(db) if a.key==country["input"]][0]
                
                # Iterate over projects within each ountry
                for project in country_ds.technosphere():
                    project_production = country_production * project["amount"]
                    
                    # If dataset is a specific project
                    if project["name"] in list(set(projects_process_contribution_df["LCI dataset"])):
                        # Get dataframe with project impacts
                        project_impacts_df = projects_process_contribution_df[(projects_process_contribution_df["LCI dataset"]==project["name"]) &
                                                                             (projects_process_contribution_df["Impact"]==impact) &
                                                                             (projects_process_contribution_df["Year"]==year)]
                        
                        # Calculate technology contribution
                        project_technology = map_lithium_projects_to_lci[map_lithium_projects_to_lci["LCI dataset"] == project["name"]]["Technology"].values[0]
                        if project_technology not in technology_impact_contribution:
                            technology_impact_contribution[project_technology] = project_production * project_impacts_df["Total"].values[0]
                        else:
                            technology_impact_contribution[project_technology] += project_production * project_impacts_df["Total"].values[0]

                        # Caculate background process contribution
                        electricity_contribution += project_production * project_impacts_df["Electricity"].values[0]
                        heat_contribution += project_production * project_impacts_df["Heat"].values[0]
                        reagent_contribution += project_production * project_impacts_df["Reagents"].values[0]     
                        foreground_contribution += project_production * project_impacts_df["Foreground"].values[0]        
                        other_contribution += project_production * project_impacts_df["Other"].values[0]                                            
                    
                    # If dataset is the country-average market:
                    else:
                        # Get country average dataset
                        other_ds = [a for a in bw.Database(db) if a.key==project["input"]][0]
                        # Iterate over projects within average dataset:
                        for other in other_ds.technosphere():
                            project_other_production = project_production * other["amount"]

                            project_impacts_df = projects_process_contribution_df[(projects_process_contribution_df["LCI dataset"]==other["name"]) &
                                                                                  (projects_process_contribution_df["Impact"]==impact) &
                                                                                  (projects_process_contribution_df["Year"]==year)]

                            # Calculate technology contribution
                            if "Modelled as average" not in technology_impact_contribution:
                                technology_impact_contribution["Modelled as average" ] = project_other_production * project_impacts_df["Total"].values[0]
                            else:
                                technology_impact_contribution["Modelled as average" ] += project_other_production * project_impacts_df["Total"].values[0]

                            electricity_contribution += project_other_production * project_impacts_df["Electricity"].values[0]
                            heat_contribution += project_other_production * project_impacts_df["Heat"].values[0]
                            reagent_contribution += project_other_production * project_impacts_df["Reagents"].values[0]     
                            foreground_contribution += project_other_production * project_impacts_df["Foreground"].values[0]        
                            other_contribution += project_other_production * project_impacts_df["Other"].values[0]      
            
            technology_contribution[(li_sc, year, impact)] = technology_impact_contribution
            process_contribution.append((li_sc, year, impact, electricity_contribution, heat_contribution, reagent_contribution, foreground_contribution, other_contribution))

# Covert to dataframes
technology_contribution_df = []
for (li_sc, year, impact), contributions in technology_contribution.items():
    for technology, value in contributions.items():
        technology_contribution_df.append({
            'Lithium scenario': li_sc,
            'Year': year,
            'Impact': impact,
            'Technology': technology,
            'Total': value
        })
technology_contribution_df = pd.DataFrame(technology_contribution_df)

process_contribution_df = pd.DataFrame(process_contribution, columns=["Lithium scenario", "Year", "Impact",
                                    "Electricity", "Heat", "Reagents", "Direct emissions", "Other"])
process_contribution_df["Total"] = process_contribution_df[["Electricity", "Heat", "Reagents", "Direct emissions", "Other"]].sum(axis=1)

# Convert total impact to Mt CO2eq or billion m3
technology_contribution_df["Total"] /= 1e9
process_contribution_df.iloc[:,3:] /= 1e9

**Impact intensity of projects with production per year and scenario**

In [21]:
impacts_intensity_projects_by_scenario = []
for impact in LCIA_METHODS:
    for li_sc in scenario_db_mapping:
        for year in [2020, 2025, 2030, 2035]:
            if year == 2020:
                db = scenario_db_mapping['S&P-Baseline']['SSP2-NDC'][2020]
            else:
                db = scenario_db_mapping[li_sc]['SSP2-NDC'][year]

            country_markets = [a for a in bw.Database(db) 
                               if a["name"]=="market for lithium carbonate, from brine, S&P forecasts" 
                               and a["location"] != "World"]
            for country_ds in country_markets:
                for proj_ds in country_ds.technosphere():
                    if proj_ds["name"] == 'market for lithium carbonate, from brine, S&P forecasts, other projects':
                        pass
                    else:
                        proj_data = projects_process_contribution_df[(projects_process_contribution_df["Impact"]==impact) &
                                                                    (projects_process_contribution_df["Year"]==year) &
                                                                    (projects_process_contribution_df["LCI dataset"]==proj_ds["name"])][["Project name", "Total"]]
                        impacts_intensity_projects_by_scenario.append((impact, li_sc, year, country_ds["location"], 
                                                                    proj_data["Project name"].values[0], proj_data["Total"].values[0]))
                        
impacts_intensity_projects_by_scenario = pd.DataFrame(impacts_intensity_projects_by_scenario, 
                                                      columns=["Impact", "Lithium scenario", "Year", "Country", "Project name", "Intensity"])

In [22]:
# Export technology and process contribution and impacts intensity for projects within scenario
technology_contribution_df.to_csv(RESULTS_PATH / f"annual_impacts_by_technology_NDC_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)
process_contribution_df.to_csv(RESULTS_PATH / f"annual_impacts_by_source_NDC_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)
impacts_intensity_projects_by_scenario.to_csv(RESULTS_PATH / f"impacts_intensity_by_project_NDC_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

# Import results for further analysis
#process_contribution_df = pd.read_csv(RESULTS_PATH / "annual_impacts_by_source_NDC_25-02-2025.csv")

### Sanity check of total impacts

In [23]:
# Check that world impact equals sum of impact by country
def calculate_total_impact(group):
    world_impact = group[group['Country'] == 'World']['Total'].sum()
    countries_sum_impact = group[group['Country'] != 'World']['Total'].sum()
    return pd.Series({'World': world_impact, 'Countries Sum': countries_sum_impact})

technology_sum_impact = technology_contribution_df.groupby(['Lithium scenario', 'Year', 'Impact'])['Total'].sum().reset_index()
source_sum_impact = process_contribution_df.groupby(['Lithium scenario', 'Year', 'Impact'])['Total'].sum().reset_index()

sanity_check_results = impacts_by_country_df[impacts_by_country_df["IAM scenario"]=="SSP2-NDC"].groupby(["Impact", 'Lithium scenario', 'Year']).apply(calculate_total_impact).reset_index()
sanity_check_results = pd.merge(sanity_check_results, technology_sum_impact, on=['Lithium scenario', 'Year', 'Impact'], how='left')
sanity_check_results = pd.merge(sanity_check_results, source_sum_impact, on=['Lithium scenario', 'Year', 'Impact'], how='left')

sanity_check_results

,Impact,Lithium scenario,Year,World,Countries Sum,Total_x,Total_y
0,Climate change,S&P-Ambitious,2020,1.883787,1.883788,1.883787,1.883787
1,Climate change,S&P-Ambitious,2025,8.284123,8.284123,8.284123,8.284123
2,Climate change,S&P-Ambitious,2030,12.398004,12.398004,12.398004,12.398004
3,Climate change,S&P-Ambitious,2035,14.824573,14.824573,14.824573,14.824573
4,Climate change,S&P-Baseline,2020,1.883787,1.883788,1.883787,1.883787
5,Climate change,S&P-Baseline,2025,4.619703,4.619703,4.619703,4.619703
6,Climate change,S&P-Baseline,2030,7.050650,7.050650,7.050650,7.050650
7,Climate change,S&P-Baseline,2035,8.477416,8.477416,8.477416,8.477416
8,Climate change,S&P-Very Ambitious,2020,1.883787,1.883788,1.883787,1.883787
9,Climate change,S&P-Very Ambitious,2025,8.666126,8.666126,8.666126,8.666126


## 4. Sensitivity analysis

**Sensitivity in 2035 to impacts by source: electricity, heat, reagents, and direct emissions**

In [24]:
# Calculate global impacts intensity varying the impacts from electricity, heating, reagents, and direct emissions (impacts per kg)
SA_impacts_by_sources = []

for impact in LCIA_METHODS:
    for li_sc in scenario_db_mapping:
        
        # Get impacts by source and production in 2035
        impacts_sources_2035_sc = process_contribution_df[(process_contribution_df["Impact"]==impact) &
                                                          (process_contribution_df["Lithium scenario"]==li_sc) &
                                                          (process_contribution_df["Year"]==2035)][["Electricity", "Heat", "Reagents", "Direct emissions", "Other"]]
        global_production_2035_sc = lithium_production_total.loc[li_sc]["2035"]

        # Calculate impacts varying each impact source intensity
        variations = np.linspace(-1, 1, 21)
        for source in ["Electricity", "Heat", "Reagents", "Direct emissions"]:
            # Get the default emissions of the analyzed source
            source_original_impacts = impacts_sources_2035_sc[source].values[0]
            
            # Get the sum of emissions for the other sources
            other_sources = [i for i in impacts_sources_2035_sc.columns if i != source]
            others_impacts = impacts_sources_2035_sc[other_sources].sum(axis=1).values[0]

            # Calculate emissions intensity varying the assessed source
            for var in variations:
                source_varied_impacts = source_original_impacts * (1 + var)
                global_intensity_2035_var = (source_varied_impacts + others_impacts) / global_production_2035_sc * 1e3
                
                SA_impacts_by_sources.append((impact, li_sc, source, var*100, global_intensity_2035_var))

SA_impacts_by_sources_df = pd.DataFrame(SA_impacts_by_sources, 
                                            columns=["Impact", 'Lithium scenario', 'Source', 'Variation', "2035 intensity var"])

In [25]:
SA_impacts_by_sources_df.to_csv(RESULTS_PATH / f"SA_impacts_sources_2035_NDC_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

**Sensitivity to the impacts intensity by country**

In [26]:
# Calculate global impacts intensity varying the country-specific intensity (kg CO2-eq/kg)
SA_country_impacts_intensity = []

for impact in LCIA_METHODS:
    for li_sc in scenario_db_mapping:
        for year in [2020, 2025, 2030, 2035]:
            
            # Get global market for lithium production
            if year == 2020:
                db = scenario_db_mapping['S&P-Baseline']['SSP2-NDC'][2020]
            else:
                db = scenario_db_mapping[li_sc]['SSP2-NDC'][year]
            glo_market = [a for a in bw.Database(db) if a["name"]=="market for lithium carbonate, from brine, S&P forecasts" 
                                                        and a["location"]=="World"][0]
            
            # Get global production, imapcts intensity, and total impacts for scenario and year
            glo_production = lithium_production_total.loc[li_sc][str(year)] * 1e6
            glo_intensity = lca_results_country[db][(glo_market.key, None)][LCIA_METHODS[impact]]
            glo_impact = glo_intensity * glo_production

            # Get country data for scenarios and year
            country_data = {}
            for country in glo_market.technosphere():
                country_production = glo_production * country["amount"]
                country_ds = [a for a in bw.Database(db) if a.key==country["input"]][0]
                impact_intensity_country = lca_results_country[db][(country_ds.key, None)][LCIA_METHODS[impact]]
                country_data.update({country["location"]: {"Production": country_production, "Intensity": impact_intensity_country}})
            country_data = pd.DataFrame(country_data).T

            # Calculate impacts varying country-specific intensity
            variations = np.linspace(-0.5, 0.5, 11)
            for i, row in country_data.iterrows():
                original_intensity = row["Intensity"]

                # Compute total sum for each variation
                for var in variations:
                    varied_intensity = original_intensity * (1 + var)
                    df_copy = country_data.copy()
                    df_copy.at[i, "Intensity"] = varied_intensity

                    glo_impact_iter = (df_copy["Production"] * df_copy["Intensity"]).sum()                    
                    glo_intensity_iter = glo_impact_iter / glo_production
                    glo_intensity_var = (glo_intensity_iter - glo_intensity) / glo_intensity *100
                      
                    SA_country_impacts_intensity.append((impact, li_sc, year, i, var*100, glo_intensity_var))


SA_country_impacts_intensity_df = pd.DataFrame(SA_country_impacts_intensity, 
                                               columns=['Impact', 'Lithium scenario', 'Year', "Country", 'Variation', "Intensity var"])


In [27]:
SA_country_impacts_intensity_df.to_csv(RESULTS_PATH / f"SA_country_impacts_intensity_NDC{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=0)

## Calculate impacts of electricity mixes

In [29]:
elec_ds = [
    ("market for electricity, high voltage, IEA", "AR"),
    ("market for electricity, high voltage, IEA", "CL"),
    ("market group for electricity, high voltage", "CN"),
    ("market group for electricity, high voltage", "World"),
]
background_db = [db for db in list(bw.databases) if "ei_cutoff_3.10_remind_SSP2" in db]

elec_mixes_ghg =[]
for em in elec_ds:
    print(em)
    for db in background_db:
        
        actv = [d for d in bw.Database(db) if d["name"]==em[0] and d["location"]==em[1]][0]

        lca = bw.LCA({actv.key: 1}, LCIA_METHODS["Climate change"])
        lca.lci()
        lca.lcia()

        print(db, lca.score)
        elec_mixes_ghg.append((em[1], db, lca.score))
    print("####################################")

('market for electricity, high voltage, IEA', 'AR')
ei_cutoff_3.10_remind_SSP2-NDC_2020_STEPS 2025-01-27 0.3265427548351503
ei_cutoff_3.10_remind_SSP2-PkBudg1150_2030_APS 2025-01-27 0.2144563437667699
ei_cutoff_3.10_remind_SSP2-PkBudg1150_2025_APS 2025-01-27 0.28483175739663213
ei_cutoff_3.10_remind_SSP2-PkBudg1150_2035_APS 2025-01-27 0.16568219224432226
ei_cutoff_3.10_remind_SSP2-NDC_2035_STEPS 2025-01-27 0.16694852999195522
ei_cutoff_3.10_remind_SSP2-NDC_2030_STEPS 2025-01-27 0.21537912500740303
ei_cutoff_3.10_remind_SSP2-NDC_2025_STEPS 2025-01-27 0.2874785257742931
####################################
('market for electricity, high voltage, IEA', 'CL')
ei_cutoff_3.10_remind_SSP2-NDC_2020_STEPS 2025-01-27 0.3813533511399866
ei_cutoff_3.10_remind_SSP2-PkBudg1150_2030_APS 2025-01-27 0.06344091832834471
ei_cutoff_3.10_remind_SSP2-PkBudg1150_2025_APS 2025-01-27 0.16053774342965407
ei_cutoff_3.10_remind_SSP2-PkBudg1150_2035_APS 2025-01-27 0.02451388130240266
ei_cutoff_3.10_remind_SSP2-NDC

In [1]:
print("AR", (0.16694852999195522 -  0.3265427548351503) /  0.3265427548351503)
print("CL", (0.08946543880030865 - 0.3813533511399866) / 0.3813533511399866)
print("CN", (0.2943292714169822 -  0.7114246254818867) /  0.7114246254818867)
print("World", (0.15532064783977093 -  0.4995395437706676) /  0.4995395437706676)

AR -0.4887391389950255
CL -0.7654001504566094
CN -0.5862818619504253
World -0.68907236718966
